# Analysis of Alabama


## Required Packages

In [10]:
# For working with graphs
import networkx as nx
# For making plots
import matplotlib.pyplot as plt
import numpy as np

# Needed for gerrchain
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import geopandas as gpd
import pandas as pd

# So everyone uses the same randomness
import random

import maup


Read the shapefile data into a geodata frame and a graph.

In [11]:
# Read shapefile data into a geo data frame and a graph
al_gdf = gpd.read_file("al_precinct_file/Alabama_precincts_with_data.shp")
al_graph = Graph.from_file("al_precinct_file/Alabama_precincts_with_data.shp", ignore_errors=True)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/adjacency.py:112: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(1805, 1814), (1776, 1787), (99, 100), (274, 1044), (687, 697), (1178, 1182), (1787, 1802), (1798, 1802), (1806, 1815), (949, 953), (369, 378), (449, 725), (1827, 1838), (1830, 1843), (1569, 1579), (709, 730), (977, 1048), (386, 952), (1580, 1588), (797, 1099), (181, 1495), (1804, 1827), (1097, 1220), (154, 166), (80, 378), (229, 367), (513, 1685), (1780, 1789), (1793, 1817), (1816, 1819), (225, 235), (687, 1074), (272, 1070), (1794, 1809), (183, 1806), (158, 172), (365, 385), (183, 1815), (457, 468), (1476, 1490), (276, 304), (105, 175), (269, 277), (1806, 1810), (77, 85), (953, 1476), (316, 1926), (310, 1855), (469, 698), (302, 1106), (469, 716), (1532, 1534), (302, 1115), (472, 1792), (1167, 1274), (1789, 1824), (1823, 1835), (702, 704), (690, 711), (501, 543), (246, 253), (1774, 1808), (1782, 1821), (545, 558), (294,

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
al_gdf

senate_columns = ["G22USSDBOY", "G22USSLSOP", "G22USSOWRI", "G22USSRBRI"]


,UNIQUE_ID,COUNTYFP,County,Precinct,G22USSDBOY,G22USSLSOP,G22USSOWRI,G22USSRBRI,OriginalPl,AdoptedPla,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,geometry
0,Jefferson-:-PREC 1010 - HUFFMAN BAPTIST CH,073,Jefferson,PREC 1010 - HUFFMAN BAPTIST CH,1864.0,39.0,1.0,447.0,5,6,10710,10363,2141,7632,49,56,0,485,"POLYGON Z ((-86.64897 33.59396 0.00000, -86.64..."
1,Jefferson-:-PREC 1020 - TOM BRADFORD PARK,073,Jefferson,PREC 1020 - TOM BRADFORD PARK,1458.0,28.0,2.0,416.0,5,5,7929,7661,1829,5557,18,50,0,207,"POLYGON Z ((-86.65986 33.64447 0.00000, -86.65..."
2,Jefferson-:-PREC 1030 - L_M_ SMITH MIDDLE,073,Jefferson,PREC 1030 - L_M_ SMITH MIDDLE,1372.0,25.0,1.0,164.0,5,6,6729,6506,698,5598,39,17,0,154,"POLYGON Z ((-86.68488 33.61943 0.00000, -86.68..."
3,Jefferson-:-PREC 1040 - BETHEL BAPTIST CHU,073,Jefferson,PREC 1040 - BETHEL BAPTIST CHU,1504.0,12.0,1.0,36.0,6,6,4095,4038,78,3943,4,1,0,12,"POLYGON Z ((-86.85019 33.55959 0.00000, -86.85..."
4,Jefferson-:-PREC 1060 - SUN VALLEY ELEMENT,073,Jefferson,PREC 1060 - SUN VALLEY ELEMENT,1472.0,12.0,4.0,206.0,5,6,5324,5188,673,4163,16,15,2,319,"POLYGON Z ((-86.70259 33.63731 0.00000, -86.70..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,Choctaw-:-TOOMEY,023,Choctaw,TOOMEY,44.0,1.0,0.0,79.0,6,6,287,279,167,110,0,1,0,1,"POLYGON Z ((-88.35069 31.74690 0.00000, -88.35..."
1935,Choctaw-:-TOXEY,023,Choctaw,TOXEY,80.0,1.0,0.0,150.0,6,6,583,564,352,209,2,0,0,1,"POLYGON Z ((-88.35634 31.95628 0.00000, -88.35..."
1936,Choctaw-:-WOMACK HILL,023,Choctaw,WOMACK HILL,15.0,0.0,0.0,71.0,6,6,138,135,108,26,0,0,0,1,"POLYGON Z ((-88.22792 31.84026 0.00000, -88.22..."
1937,Choctaw-:-YANTLEY,023,Choctaw,YANTLEY,154.0,2.0,0.0,19.0,6,6,445,434,61,368,3,0,0,2,"POLYGON Z ((-88.30748 32.21943 0.00000, -88.30..."
